# A. EDINET APIを用いてXBRLデータを取得する
'Dropbox\TSR_RA_Ochiai\Note\EDINET\EDINET_API仕様書.pdf'を参照

## 1. 書類一覧APIから検索する書類を抽出する
エンドポイント：'https://disclosure.edinet-fsa.go.jp/api/v1/documents.json'<br>
HTTPメソッド：GETメソッド<br>
バージョン：'v1'を使用<br>
リクエストパラメータ：
- date：YYYY-MM-DDで日付を指定
- type：1→メタデータのみ、2→提出書類一覧及びメタデータを取得

In [35]:
import pandas as pd
import requests
import datetime
import zipfile
import glob
import os
import pickle
from arelle import ModelManager
from arelle import Cntlr

### 期間を指定する関数

In [2]:
def make_day_list(start_date, end_date):
    """開始日時と終了日時を設定すると、期間内の日付がすべて入ったリストが返される

    Args:
        start_date (datetime.date): 開始日時
        end_date (datetime.date): 終了日時

    Returns:
        list: 開始日時から終了日時までのすべての日付が格納されたリスト
    """
    period = int((end_date - start_date).days)
    day_list = []
    for d in range(period):
        day = start_date + datetime.timedelta(days=d)
        day_list.append(day)
    
    day_list.append(end_date)
    print(f'start_date is {day_list[0]}')
    print(f'end_date is {day_list[-1]}')
    print(f'period is {period} days')

    return day_list

### データを取得する期間を設定し、該当期間に提出された書類一覧(ID)を取得する

In [66]:
def save_doc_id(start_date, end_date, fc, oc, path):
    """書類情報を取得する

    Args:
        start_date (datetime.date): start date
        end_date (datetime.date): end date
        fc (str): formCode (様式コード)
        oc (str): ordinanceCode (府令コード)
        path (str): pickleオブジェクト保存先のパス
    """
    # データを取得する期間の設定
    day_list = make_day_list(start_date, end_date)

    # メタデータと提出書類一覧をJSON形式で取得
    id_list = []
    for day in day_list:
        print(day)
        params = {'type': 2, 'date': day}
        url = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents.json'
        meta_data = requests.get(url=url, params=params)
        if meta_data.status_code == 200:
            meta_data_json = meta_data.json()
            meta_data_df = pd.json_normalize(meta_data_json)
            c = meta_data_df['metadata.resultset.count']
            if not int(c) == 0:
                results = pd.DataFrame(meta_data_json['results'])
                results = results[(results['ordinanceCode'] == oc) & (results['formCode'] == fc)]
                id = list(results['docID'])
                id_list.append(id)

    with open(path, mode='wb') as file:
        pickle.dump(id_list, file)

In [67]:
start_date = datetime.date(2017, 1, 1)
end_date = datetime.date(2021, 9, 15)
path = 'C:/Users/koeci/Google ドライブ/MBA/ワークショップ/data/EDINET/download_list/download_list_new_release.pickle'
save_doc_id(start_date, end_date, fc='024000', oc='010', path=path)

start_date is 2017-01-01
end_date is 2021-09-15
period is 1718 days
2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-04
2017-03-05
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-11
2017-03-12
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-25
2017-03-

## 2. 書類取得APIからXBRLデータを取得する
エンドポイント：'https://disclosure.edinet-fsa.go.jp/api/v1/documents/書類管理番号'  
HTTPSメソッド：GETメソッド  
バージョン：'v1'を使用  
リクエストパラメータ：
- type：1→提出本文書及び監査報告書(XBRLファイルを含む)、2→PDFを取得、3→代替書面・添付文書を取得、4→英文ファイルを取得

### 試しに、１社のみでzipファイルをダウンロードしてみる

In [63]:
# 1社で試しにダウンロードしてみる→zipファイルがダウンロードされる
id = docID_list[0]
params = {'type': 1}
url = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents/' + id
doc = requests.get(url=url, params=params, stream=True)

filename = '../../Data/EDINET/API_data/' + id + '.zip'
if doc.status_code == 200:
    with open(file=filename, mode='wb') as file:
        file.write(doc.content)

### １で取得したIDのリストを元に、すべての企業・すべての期間のzipファイルをダウンロードする

In [68]:
def download_zip(pickle_path, save_path, start=0):
    import time

    # 書類管理番号を取得する
    with open(pickle_path, mode='rb') as file:
        id_list = pickle.load(file)
    docID_list = [k for i in id_list for k in i]
    docID_list = docID_list[start:]

    # 書類管理番号ごとに書類を取得する
    params = {'type': 1}
    sum_all = len(docID_list)

    for i, id in enumerate(docID_list):
        print(f'{i + 1} / {sum_all} was saved.')
        url = 'https://disclosure.edinet-fsa.go.jp/api/v1/documents/' + id
        doc = requests.get(url=url, params=params)
        filename = save_path + id + '.zip'
        # HTTPリクエストが正しく処理された場合にファイルの作成を実行
        if doc.status_code == 200:
            with open(filename, mode='wb') as file:
                content = doc.content
                file.write(content)
        time.sleep(0.6)

In [69]:
pickle_path = 'C:/Users/koeci/Google ドライブ/MBA/ワークショップ/data/EDINET/download_list/download_list_new_release.pickle'
save_path = 'D:/Workshop_Data/new_release/'
download_zip(pickle_path=pickle_path, save_path=save_path, start=0)

1 / 439 was saved.
2 / 439 was saved.
3 / 439 was saved.
4 / 439 was saved.
5 / 439 was saved.
6 / 439 was saved.
7 / 439 was saved.
8 / 439 was saved.
9 / 439 was saved.
10 / 439 was saved.
11 / 439 was saved.
12 / 439 was saved.
13 / 439 was saved.
14 / 439 was saved.
15 / 439 was saved.
16 / 439 was saved.
17 / 439 was saved.
18 / 439 was saved.
19 / 439 was saved.
20 / 439 was saved.
21 / 439 was saved.
22 / 439 was saved.
23 / 439 was saved.
24 / 439 was saved.
25 / 439 was saved.
26 / 439 was saved.
27 / 439 was saved.
28 / 439 was saved.
29 / 439 was saved.
30 / 439 was saved.
31 / 439 was saved.
32 / 439 was saved.
33 / 439 was saved.
34 / 439 was saved.
35 / 439 was saved.
36 / 439 was saved.
37 / 439 was saved.
38 / 439 was saved.
39 / 439 was saved.
40 / 439 was saved.
41 / 439 was saved.
42 / 439 was saved.
43 / 439 was saved.
44 / 439 was saved.
45 / 439 was saved.
46 / 439 was saved.
47 / 439 was saved.
48 / 439 was saved.
49 / 439 was saved.
50 / 439 was saved.
51 / 439 

### 途中で処理落ちした時は、以下のセルで"start"にエラーしたファイル番号を入力して実行する
ファイル番号は上のセルで順次出力されるため、最後の番号を入力すればよい

# B. 取得したXBRLデータから財務情報を抽出する

参考：https://qiita.com/XBRLJapan/items/b1e66f79d597df7b6037

## 1. zipファイルの解凍

In [42]:
# まずファイルの中身を確認する
with zipfile.ZipFile(zip_path, 'r') as file:
    file_list = file.namelist()
print(file_list)

PermissionError: [Errno 13] Permission denied: 'D:/Workshop_Data/securities/'

In [41]:
def unzip(zip_path, xbrl_path):
    for i, f in enumerate(zip_files):
        len_zip_files = len(zip_files)
        print(f'file:{f}, loading {i+1} / {len_zip_files}')
        with zipfile.ZipFile(f) as zip_f:
            zip_f.extractall(zip_path)
        
    xbrl_files = glob.glob(xbrl_path)
    return xbrl_files

In [46]:
zip_path = 'D:/Workshop_Data/securities/'
xbrl_path = 'D:/Workshop_Data/securities/XBRL/PublicDoc/*.xbrl'
zip_files = glob.glob(os.path.join(zip_path, '*.zip'))

# 以下のコマンドでzipファイルがすべて展開されるので実行には注意
xbrl_files = unzip(zip_path, xbrl_path)

file:D:/Workshop_Data/securities\S1009E6F.zip, loading 1 / 20876
file:D:/Workshop_Data/securities\S1009E9Q.zip, loading 2 / 20876
file:D:/Workshop_Data/securities\S1009FVE.zip, loading 3 / 20876
file:D:/Workshop_Data/securities\S1009FVN.zip, loading 4 / 20876
file:D:/Workshop_Data/securities\S1009GFP.zip, loading 5 / 20876
file:D:/Workshop_Data/securities\S1009GPB.zip, loading 6 / 20876
file:D:/Workshop_Data/securities\S1009GWB.zip, loading 7 / 20876
file:D:/Workshop_Data/securities\S1009H96.zip, loading 8 / 20876
file:D:/Workshop_Data/securities\S1009HAE.zip, loading 9 / 20876
file:D:/Workshop_Data/securities\S1009HCF.zip, loading 10 / 20876
file:D:/Workshop_Data/securities\S1009HF1.zip, loading 11 / 20876
file:D:/Workshop_Data/securities\S1009HFU.zip, loading 12 / 20876
file:D:/Workshop_Data/securities\S1009HHQ.zip, loading 13 / 20876
file:D:/Workshop_Data/securities\S1009HI8.zip, loading 14 / 20876
file:D:/Workshop_Data/securities\S1009HL6.zip, loading 15 / 20876
file:D:/Workshop_Da

In [ ]:
# 展開せずにxbrlファイルのパスを取得する場合は以下を実行する
xbrl_files = glob.glob(xbrl_path)

## 2. XBRLデータから財務情報を抽出

各フォルダパスの設定

In [21]:
code_list_path = '../../Data/EDINET/code_list/EdinetcodeDlInfo.csv'
zip_dir = '../../Data/EDINET/API_data/'
xbrl_path = '../../Data/EDINET/API_data/XBRL/PublicDoc/*.xbrl'
csv_dir = '../../Data/EDINET/csv_data/'

XBRLデータの読み込み（処理が非常に重いので注意：１年でおよそ４時間）

In [13]:
def make_edinet_info_list(path):
    edinet_info = pd.read_csv(path, skiprows=1, encoding='cp932')
    edinet_info = edinet_info[['ＥＤＩＮＥＴコード', '提出者業種']]
    edinet_info_list = edinet_info.values.tolist()
    return edinet_info_list

edinet_info_list = make_edinet_info_list(code_list_path)

EDINETコードリストから、コードと業種を取得  
補助金の要素名：SubsidyIncomeNOI

In [33]:
all_company_info_list = []
for i, xbrl_file in enumerate(xbrl_files):
    edinet_code = ''
    firm_name = ''
    industry_code = ''
    subsidy = ''
    company_info_list = []

    ctrl = Cntlr.Cntlr()
    model_manager = ModelManager.initialize(ctrl)
    model_xbrl = model_manager.load(xbrl_file)

    for fact in model_xbrl.facts:
        if fact.concept.qname.localName == 'EDINETCodeDEI':
            print(f'EDINETコード：{fact.value}')
            edinet_code = fact.value

            for code_name in edinet_info_list:
                if code_name[0] == edinet_code:
                    print(f'業種：{code_name[1]}')
                    industry_code = code_name[1]
                    break
        
        elif fact.concept.qname.localName == 'FilerNameInJapaneseDEI':
            print(f'企業名：{fact.value}')
            firm_name = fact.value
        elif fact.concept.qname.localName == 'SubsidyIncomeNOI':
            print(f'補助金：{fact.value}')
            subsidy = fact.value

    company_info_list.append(edinet_code)
    company_info_list.append(firm_name)
    company_info_list.append(industry_code)
    company_info_list.append(subsidy)
        
    all_company_info_list.append(company_info_list)

EDINETコード：E00004
業種：水産・農林業
企業名：カネコ種苗株式会社
EDINETコード：E00006
業種：水産・農林業
企業名：株式会社　サカタのタネ
EDINETコード：E00008
業種：水産・農林業
企業名：ホクト株式会社
補助金：4000000
補助金：19000000
EDINETコード：E00009
業種：水産・農林業
企業名：株式会社アクシーズ
EDINETコード：E00010
業種：水産・農林業
企業名：株式会社ホーブ
EDINETコード：E00011
業種：建設業
企業名：住友林業株式会社
補助金：338000000
補助金：545000000
補助金：178000000
補助金：
EDINETコード：E00012
業種：水産・農林業
企業名：株式会社極洋
EDINETコード：E00014
業種：水産・農林業
企業名：日本水産株式会社
EDINETコード：E00015
業種：水産・農林業
企業名：マルハニチロ株式会社
EDINETコード：E00017
業種：卸売業
企業名：株式会社ホウスイ
EDINETコード：E00021
業種：非鉄金属
企業名：三菱マテリアル株式会社
EDINETコード：E00023
業種：非鉄金属
企業名：住友金属鉱山株式会社
EDINETコード：E00024
業種：非鉄金属
企業名：三井金属鉱業株式会社
EDINETコード：E00026
業種：非鉄金属
企業名：日本精鉱株式会社
EDINETコード：E00027
業種：非鉄金属
企業名：中外鉱業株式会社
EDINETコード：E00028
業種：非鉄金属
企業名：ＤＯＷＡホールディングス株式会社
EDINETコード：E00030
業種：石油・石炭製品
企業名：日本コークス工業株式会社
EDINETコード：E00032
業種：非鉄金属
企業名：古河機械金属株式会社
EDINETコード：E00033
業種：サービス業
企業名：常磐興産株式会社
EDINETコード：E00034
業種：卸売業
企業名：太平洋興発株式会社
補助金：14297000
補助金：19500000
EDINETコード：E00035
業種：機械
企業名：株式会社ニッチツ
EDINETコード：E00036
業種：鉱業
企業名：日鉄鉱業株式会社
EDINETコード：E00037
業種：鉱業
企業名：三

In [47]:
df = pd.DataFrame(all_company_info_list, \
    columns=['EDINET_code', 'Firm_name', 'Industory', 'Subsidy'])
df.set_index('EDINET_code', inplace=True)
df = df.query("Subsidy != ''")
# df.dropna(subset=['Subsidy'], inplace=True)

In [48]:
df.to_csv('../../Data/EDINET/csv_data/subsidy_2020.csv', \
    encoding='cp932', header=True, index=True)

In [49]:
with open('all_company_info_list', 'wb') as file:
    pickle.dump(all_company_info_list, file)

with open('all_company_info_list', 'rb') as file:
    a = pickle.load(file)